In [6]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('../AirBnb datasets/barcelona/listings.csv')

listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,269467,https://www.airbnb.com/rooms/269467,20240615011618,2024-06-15,city scrape,Sunny and quiet Loft next to Sagrada Familia,The apartment is a cosy open space with 65 m2....,NaN,https://a0.muscache.com/pictures/a6445a90-94eb...,1411775,https://www.airbnb.com/users/show/1411775,Jeremies,2011-11-16,"Barcelona, Spain","We are Ok Apartment Barcelona, a local agency ...",within an hour,100%,88%,f,https://a0.muscache.com/im/pictures/user/1f974...,https://a0.muscache.com/im/pictures/user/1f974...,La Sagrada Família,5.0,7.0,"['email', 'phone']",t,t,NaN,la Sagrada Família,Eixample,41.401810,2.17610,Entire loft,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",$176.00,7,760,7,7,760,760,7.0,760.0,NaN,t,0,0,0,257,2024-06-15,90,17,1,2018-01-01,2024-05-22,4.74,4.87,4.59,4.83,4.79,4.81,4.67,HUTB-001129,t,3,3,0,0,1.15
1,270197,https://www.airbnb.com/rooms/270197,20240615011618,2024-06-15,city scrape,*monthly stays* 4 Bedroom Apartment near Rambla,Rented only for monthly periods. 4 bedroom apa...,NaN,https://a0.muscache.com/pictures/69907440-d8df...,1414702,https://www.airbnb.com/users/show/1414702,Ferran,2011-11-16,"Barcelona, Spain","Hello, everyone!\n\nBorn and bred in Barcelona...",within an hour,98%,97%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,El Raval,36.0,112.0,"['email', 'phone']",t,t,NaN,el Raval,Ciutat Vella,41.379360,2.17376,Entire rental unit,Entire home/apt,6,1.5,1.5 baths,4.0,5.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",$74.00,32,760,32,32,760,760,32.0,760.0,NaN,t,0,0,0,0,2024-06-15,2,0,0,2016-07-03,2016-08-13,4.00,5.00,4.50,5.00,5.00,5.00,4.50,N/A SOLO ESTANCIAS MENSUALES,f,27,27,0,0,0.02
2,18674,https://www.airbnb.com/rooms/18674,20240615011618,2024-06-15,city scrape,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,https://a0.muscache.com/pictures/13031453/413c...,71615,https://www.airbnb.com/users/show/71615,Mireia Maria,2010-01-19,"Barcelona, Spain","We are Mireia (47) & Maria (49), two multiling...",within an hour,95%,85%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,la Sagrada Família,41.0,45.0,"['email', 'phone']",t,t,"Barcelona, CT, Spain",la Sagrada Família,Eixample,41.405560,2.17262,Entire rental unit,Entire home/apt,8,2.0,2 baths,3.0,6.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",$257.00,1,1125,1,4,1125,1125,2.1,1125.0,NaN,t,10,36,66,177,2024-06-15,41,5,0,2013-05-27,2024-05-02,4.35,4.45,4.63,4.75,4.68,4.78,4.30,HUTB-002062,t,27,27,

In [7]:
# Identify columns where all values are the same
constant_columns = [col for col in listings.columns if listings[col].nunique() == 1]

# Print the columns where all values are the same
print("Columns where all values are the same:", constant_columns)

Columns where all values are the same: ['scrape_id', 'last_scraped', 'calendar_last_scraped']


In [8]:
# Remove columns where all values are the same
columns_to_remove = ['scrape_id', 'last_scraped', 'calendar_last_scraped']
listings = listings.drop(columns=columns_to_remove)

In [9]:
# Remove redundant or uninformative features
columns_to_remove = ['listing_url', 'source', 'name', 'description', 'neighborhood_overview',
                     'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                     'host_picture_url', 'license', 'calendar_updated', 'host_location', 'host_neighbourhood',
                     'host_total_listings_count', 'host_verifications', 'neighbourhood', 
                     'neighbourhood_cleansed', 'bathrooms_text', 'minimum_minimum_nights', 
                     'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                     'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 
                     'availability_60', 'availability_90', 'availability_365', 'calculated_host_listings_count',
                     'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
                     'calculated_host_listings_count_shared_rooms', 'latitude', 'longitude']

listings = listings.drop(columns=columns_to_remove)

In [12]:
# Convert boolean values into binary representations
listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f': 0})
listings['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f': 0})

In [ ]:
# Remove the dollar symbol and commas, then convert the column to numeric
listings['price'] = listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float)

In [ ]:
# Handle Date columns (convert to datetime, then to numeric)
if 'host_since' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['host_since'] = pd.to_datetime(listings['host_since'])
    listings['host_since'] = listings['host_since'].astype(int) // 10**9  # Convert to Unix timestamp

if 'first_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['first_review'] = pd.to_datetime(listings['first_review'])
    listings['first_review'] = listings['first_review'].astype(int) // 10**9  # Convert to Unix timestamp

if 'last_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['last_review'] = pd.to_datetime(listings['last_review'])
    listings['last_review'] = listings['last_review'].astype(int) // 10**9  # Convert to Unix timestamp

Importing amenities from "amenities.csv" dataset

In [ ]:
amenities_dummies = pd.read_csv('amenities.csv')

In [ ]:
# Concatenate the original DataFrame (excluding the old 'amenities' column) with the new boolean features
listings = pd.concat([listings.drop(columns=['amenities']), amenities_dummies], axis=1)

In [ ]:
sentiment = pd.read_csv('sentiment.csv')

In [ ]:
listings['id'] = listings['id'].astype(str)
sentiment['listing_id'] = sentiment['listing_id'].astype(str)

# Merge the datasets on the corresponding ID columns
merged_data = pd.merge(listings, sentiment, left_on='id', right_on='listing_id', how='left')

# Drop the duplicate 'listing_id' column if it exists
merged_data = merged_data.drop(columns=['listing_id', 'id'])

merged_data

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Identify numeric and categorical columns
numeric_features = merged_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = merged_data.select_dtypes(include=['object', 'category']).columns

# Encode categorical features
encoder = OneHotEncoder(drop='first', sparse=False)
listings_encoded = pd.DataFrame(encoder.fit_transform(merged_data[categorical_features]), columns=encoder.get_feature_names_out(categorical_features), index=merged_data.index)

# Drop original categorical columns and concatenate the encoded ones
listings = merged_data.drop(columns=categorical_features)
listings = pd.concat([listings, listings_encoded], axis=1)

listings

Missing values handling through Regression Imputation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Function to fill missing values using linear regression
def fill_missing_values(df):
    df_filled = df.copy()
    
    for column in df.columns:
        if df[column].isnull().sum() > 0:

            # Separate the data into columns with and without missing values
            df_not_null = df_filled[df_filled[column].notnull()]
            df_null = df_filled[df_filled[column].isnull()]
            
            if df_not_null.empty or df_null.empty:
                # Skip column if there are no missing values
                continue
            
            # Define the model
            model = LinearRegression()
            
            # Ensure that X_train and X_test have no missing values
            X_train = df_not_null.drop(columns=[column])
            y_train = df_not_null[column]
            
            # Check for any NaNs in the training data
            if X_train.isnull().any().any():
                # If training data for the column contains NaNs, imput missing values before training
                imputer = SimpleImputer(strategy='mean')
                X_train = imputer.fit_transform(X_train)
            
            X_test = df_null.drop(columns=[column])
            
            if X_test.isnull().any().any():
                # If test data for the column contains NaNs, imput missing values before prediction
                X_test = imputer.transform(X_test)
            
            # Train the model
            model.fit(X_train, y_train)
            
            # Predict the missing values
            predicted_values = model.predict(X_test)
            
            # Assign the predicted values to the original dataframe
            df_filled.loc[df_filled[column].isnull(), column] = predicted_values
    
    return df_filled


# Apply the function to fill missing values in the numeric data
listings_filled = fill_missing_values(listings)

In [ ]:
listings_filled

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the numeric data
scaler = StandardScaler()
listings_scaled = scaler.fit_transform(listings_filled)

# Convert the scaled data back to a DataFrame
listings = pd.DataFrame(listings_scaled, columns=listings_filled.columns)

listings

<h3>Feature selection with Lasso Regularization</h3>

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Separate features and target
X = listings.drop(columns=["price"])
y = listings["price"]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and fit Lasso model
lasso = Lasso(alpha=0.01)  # alpha is the regularization strength
lasso.fit(X_train_scaled, y_train)

# Make predictions
y_pred = lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# View the coefficients
print("Lasso Coefficients:", lasso.coef_)

In [ ]:
import numpy as np

# Get the feature names if you have them
feature_names = np.array(X.columns)  # if X is a DataFrame

# Get the coefficients from the Lasso model
coefficients = lasso.coef_

# Identify the features with non-zero coefficients
selected_features = feature_names[coefficients != 0]

print("Selected Features:", selected_features)

In [ ]:
X_selected = X[selected_features]

print("Number of features in updated dataset:", X_selected.shape[1])


In [ ]:
X_selected.to_csv('X.csv', index=False)

y.to_csv('y.csv', index=False)